In [55]:
version_all = ["PHOENIX", "XX", "OLD"]
songtype_all = ["Arcade", "Remix", "Full Song", "Short Cut"]

def DA_0():
    return [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
def DA_l():
    return [[[],[],[]],[[],[],[]],[[],[],[]],[[],[],[]]]

def songtype_to_int(songtype):
    if songtype == "Arcade":
        return 0
    elif songtype == "Remix":
        return 1
    elif songtype == "Full Song":
        return 2
    elif songtype == "Short Cut":
        return 3
    else:
        return -1

def version_to_int(version):
    if version == "PHOENIX":
        return 0
    elif version == "XX":
        return 1
    else:
        return 2

def restrict_level_range(mode, levels):
    ans = [0,0]
    ans[0] = levels[0]
    ans[1] = levels[1]
    #print(levels)
    if mode == "Single":
        if levels[1] > 26: ans[1] = 26
    else:
        if levels[1] > 28: ans[1] = 28
    if levels[0] < 20:
        ans[0] = 20
    if levels[0] > levels[1]:
        return [-1,-1]
    return ans

def add_dict(a, b):
    for item in b:
        if item in a:
            a[item] += b[item]
        else:
            a[item] = b[item]
    return a

def get_song_version_int_list(songtype, version):
    songtypeint = []
    for item in songtype:
        songtypeint.append(songtype_to_int(item))
    versionint = []
    for item in version:
        versionint.append(version_to_int(item))
    return songtypeint, versionint

def filter_total_count_unit(total_count, mode, levels =[20,28], songtype = songtype_all, version = version_all):
    levels = restrict_level_range(mode, levels)
    if levels[0] == -1:
        print("Error in level range")
        return {}
    count = {key: 0 for key in range(levels[0], levels[1]+1)}
    #print(count)
    songtypeint, versionint = get_song_version_int_list(songtype, version)
    #print(data["single_counts"])
    for level in range(levels[0], levels[1]+1):
        str_level = str(level)
        for songtype in songtypeint:
            for version in versionint:
                if mode == "Single":
                    #print(count[level])
                    #print(data["single_counts"][str_level][songtype][version])
                    count[level] += total_count["Single"][str_level][songtype][version]
                else:
                    count[level] += total_count["Double"][str_level][songtype][version]
    return count

def filter_total_count(total_count, mode = "Full", levels =[20,28], songtype = songtype_all, version = version_all):
    if mode == "Full":
        dicta = filter_total_count_unit(total_count, "Single", levels, songtype, version)
        dictb = filter_total_count_unit(total_count, "Double", levels, songtype, version)
        return add_dict(dicta, dictb)
    else:
        return filter_total_count_unit(total_count, mode, levels, songtype, version)

def filter_rankdata_unit(data, mode, levels =[20,28], songtype = songtype_all, version = version_all):

    levels = restrict_level_range(mode, levels)
    if levels[0] == -1:
        print("Error in level range")
        return {}
    filtered_rank = {key: [] for key in range(levels[0], levels[1]+1)}

    songtypeint, versionint = get_song_version_int_list(songtype, version)
    #print(songtypeint, versionint)
    for level in range(levels[0], levels[1]+1):
        str_level = str(level)
        for songtype in songtypeint:
            for version in versionint:
                    if mode == "Single":
                        for item in data["single_data"][str_level][songtype][version]:
                            filtered_rank[level].append(item)
                    else:
                        for item in data["double_data"][str_level][songtype][version]:
                            filtered_rank[level].append(item) 
    return filtered_rank

def filter_rankdata(data, mode = "Full", levels = [20,28], songtype = songtype_all, version = version_all):
    if mode == "Full":
        dicta = filter_rankdata_unit(data, "Single", levels, songtype, version)
        dictb = filter_rankdata_unit(data, "Double", levels, songtype, version)
        return add_dict(dicta, dictb)
    else:
        return filter_rankdata_unit(data, mode, levels, songtype, version)

def filter_rankcount_unit(data, mode, levels = [20,28], songtype = songtype_all, version = version_all):
    levels = restrict_level_range(mode, levels)

    if levels[0] == -1:
        print("Error in level range")
        return {}
    count = {key: 0 for key in range(levels[0], levels[1]+1)}
    #print(count)
    songtypeint, versionint = get_song_version_int_list(songtype, version)
    #print(data["single_counts"])
    for level in range(levels[0], levels[1]+1):
        str_level = str(level)
        for songtype in songtypeint:
            for version in versionint:
                if mode == "Single":
                    #print(count[level])
                    #print(data["single_counts"][str_level][songtype][version])
                    count[level] += data["single_counts"][str_level][songtype][version]
                else:
                    count[level] += data["double_counts"][str_level][songtype][version]
    return count

def filter_rankcount(data, mode = "Full", levels = [20,28], songtype = songtype_all, version = version_all):
    if mode == "Full":
        dicta = filter_rankcount_unit(data, "Single", levels, songtype, version)
        dictb = filter_rankcount_unit(data, "Double", levels, songtype, version)
        return add_dict(dicta, dictb)
    else:
        return filter_rankcount_unit(data, mode, levels, songtype, version)
    

In [3]:
import json
user_ranks = json.load(open("user_rankdata.json"))

total_steps_count = json.load(open("total_steps_count.json"))

duplicate_users = json.load(open("duplicate_users.json"))


In [54]:
#level_weight = {20: 8, 21: 10, 22: 14, 23: 22, 24: 38, 25: 70, 26: 134, 27: 262, 28: 518}
level_weight = {20: 8, 21: 10, 22: 14, 23: 20, 24: 28, 25: 38, 26: 50, 27: 64, 28: 70}
rank_weight = lambda x: (200-x)/200



def rankdata_unit_str(data):
    mode = "D"
    if data["mode"] == "Single": mode = "S"

    return data["song"] + " " + mode + str(data["level"]) + ": rank " + str(data["rank"]) + ", score: " + str(data["score"])
    
def total_count(data):
    total = 0
    for level in data:
        total += data[level]
    return total

def total_count_weighted(data):
    total = 0
    for level in data:
        total += data[level]*level_weight[level]
    return total

def total_count_with_rank(data):
    total = 0
    for level in data:
        for ranks in range(len(data[level])):
            total += level*rank_weight(data[level][ranks]["rank"])
    return total

def total_count_with_rank_weighted(data):
    total = 0
    for level in data:
        for ranks in range(len(data[level])):
            total += level*rank_weight(data[level][ranks]["rank"])*level_weight[level]
    return total

def sort_rankdata(data, sort_key = "score", sort_all = False):
    if sort_all == False:
        for level in data:
            if sort_key == "score":
                data[level].sort(key = lambda x: x['score'], reverse = True)
            elif sort_key == "rank":
                data[level].sort(key = lambda x: x['rank'])
        total_list = []
        for level in sorted(data.keys(), reverse= True):
            total_list += data[level]
    else:
        total_list = []
        for level in sorted(data.keys(), reverse= True):
            total_list += data[level]
        if sort_key == "score":
            total_list.sort(key = lambda x: x['score'], reverse = True)
        elif sort_key == "rank":
            total_list.sort(key = lambda x: x['rank'])
    return total_list


In [5]:

frankeza = sort_rankdata(filter_rankdata(user_ranks[1],"Full", [25,27],songtype_all, version_all), "rank", True)


sortme = []
for i in range(300):
    sortme.append([user_ranks[i]["username"], total_count_with_rank_weighted(filter_rankdata(user_ranks[i], levels= [15,28]))])

sortme.sort(key = lambda x: x[1], reverse = True)
for i in range(30):
    print(sortme[i][0]+": "+str(sortme[i][1]))


HAPPYF333TZ: 312415.3200000019
FRANKEZA: 299322.3500000013
JERRYWHYMARI: 256168.96000000025
FEFEMZ: 234655.93999999992
MATTMILLER: 229030.65999999997
RUN: 227230.5700000001
NATURENIM: 209362.40000000002
LAILAS: 204369.64
ZELLLOOO: 203948.08000000013
JQTRAN: 203469.83000000005
KANATA: 202744.73000000019
CHOISUN: 200806.93999999986
KANATA: 197966.19999999998
SANCTUS: 197633.79999999967
URUSA: 194292.15999999997
SUNMU: 191170.59999999998
RSS: 190495.51999999984
SEOLJIHO: 186942.00999999995
GORANI: 179550.90999999997
FR0D0XXNVT: 177824.36999999994
SUNNY: 175641.49
LHA: 174950.9000000001
KANATA: 174750.78999999995
SANGKM: 169734.8299999999
YEIDEN: 169035.14999999988
KIMSH: 165507.00999999998
TOMATONIUM: 157595.25999999998
ZZANGK: 155648.37000000005
REY: 155397.50000000006
HKPIU0127: 150325.26999999987


In [6]:
'''
#중복된 유저명이 있는지 확인
def get_duplicate_usernames_dict(user_set):
    user_dict = {}
    for user in user_set:
        username = user["username"]
        if username not in user_dict:
            user_dict[username] = [user["userID"]]
        else:
            user_dict[username].append(user["userID"])

    filtered_dict = {k: v for k, v in user_dict.items() if len(v) > 1}
    return filtered_dict

duplicateIDs = get_duplicate_usernames_dict(user_ranks)
#json.dump(duplicateIDs, open("duplicate_users.json", "w"))
'''

'\n#중복된 유저명이 있는지 확인\ndef get_duplicate_usernames_dict(user_set):\n    user_dict = {}\n    for user in user_set:\n        username = user["username"]\n        if username not in user_dict:\n            user_dict[username] = [user["userID"]]\n        else:\n            user_dict[username].append(user["userID"])\n\n    filtered_dict = {k: v for k, v in user_dict.items() if len(v) > 1}\n    return filtered_dict\n\nduplicateIDs = get_duplicate_usernames_dict(user_ranks)\n#json.dump(duplicateIDs, open("duplicate_users.json", "w"))\n'

In [7]:
def search_user(username, exact = False):
    usernames = {}   
    search_term = username.upper()

    # Iterate through each user in the user ranks list
    for user in user_ranks:
        # Check if the search term is in the current username in the list
        if search_term in user["username"].upper():
            cleardata = filter_rankcount(user)
            # Add or append the userID to the usernames dictionary
            if exact == False:
                if user["username"] not in usernames:
                    usernames[user["username"]] = [[user["userID"], cleardata]]
                else:
                    usernames[user["username"]].append([user["userID"], cleardata])
            else:
                if user["username"] == search_term:
                    if user["username"] not in usernames:
                        usernames[user["username"]] = [[user["userID"], cleardata]]
                    else:
                        usernames[user["username"]].append([user["userID"], cleardata])
            
    return usernames

def print_search_user(username, exact = False):
    strs = []
    printme = search_user(username, exact)
    for user in printme:
        strs.append(user)
        for item in printme[user]:
            strs.append(item[0]+": "+str(item[1]))
        strs.append("")
    return strs

def return_user(username, userID):
    if len(userID) == 4:
        userID = "#" + str(userID)
    for user in user_ranks:
        if user["username"] == username and user["userID"] == userID:
            return user 
    return None


def return_user_with_name(username):
    username_upper = username.upper()
    if username_upper in duplicate_users:
        str1 = "Duplicate users found"
        str2 = "Please search with the following format: [username] [userID]"
        str3 = "The First User is selected among: \n"
        str4 = print_search_user(username_upper, exact = True)
        print(str1)
        print(str2)
        print(str3)
        for item in str4:
            print(item)
        userID = search_user(username_upper, exact = True)[username_upper][0][0]
        return return_user(username_upper, userID)
    if " " in username:
        username, userID = username.split(" ")
        username_upper = username.upper()
    else:
        userID = search_user(username_upper, exact = True)[username_upper][0][0]
    return return_user(username_upper, userID)


{'key': 'EXC #7437',
 'username': 'EXC',
 'userID': '#7437',
 'single_data': {'20': [[[], [], []],
   [[], [], []],
   [[], [], []],
   [[], [], []]],
  '21': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]],
  '22': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]],
  '23': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]],
  '24': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]],
  '25': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]],
  '26': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]]},
 'single_counts': {'20': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
  '21': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
  '22': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
  '23': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
  '24': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
  '25': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
  '26': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]},
 'double_data': {'20': [[[], [], []],
   [[], [], []],
 

In [8]:
def rankdata_raw(username, mode = "Full", levels = [20,28], songtype = songtype_all, version = version_all):
    user = return_user_with_name(username)
    
    userCount = filter_rankcount(user, mode, levels, songtype, version)
    userData = filter_rankdata(user, mode, levels, songtype, version)
    total_filtered = filter_total_count(total_steps_count, mode, levels, songtype, version)
    return userData, userCount, total_filtered
    #

def rankdata_compare_raw(userAname, userBname, mode = "Full", levels = [20,28], songtype = songtype_all, version = version_all):
    userA = return_user_with_name(userAname)
    userB = return_user_with_name(userBname)
    userAcount = filter_rankcount(userA, mode, levels, songtype, version)
    userBcount = filter_rankcount(userB, mode, levels, songtype, version)
    userAdata = filter_rankdata(userA, mode, levels, songtype, version)
    userBdata = filter_rankdata(userB, mode, levels, songtype, version)
    total_filtered = filter_total_count(total_steps_count, mode, levels, songtype, version)
    return userAdata, userBdata, userAcount, userBcount, total_filtered



In [52]:
import pandas as pd

def color_rows(row, userAkey):
    # Use RGBA for semi-transparent colors (the last number is the alpha value)
    # The alpha value can be adjusted between 0 (fully transparent) and 1 (fully opaque)
    # Here we set it to 0.5 to blend the color with the white background, making it lighter
    color = 'rgba(125, 50, 50, 0.5)' if row['Winner'] == userAkey else 'rgba(50,50,125, 0.5)'
    return ['background-color: ' + color] * len(row)


def aggregate_dataframe(userA_data, userB_data, userA, userB):
    aggregated_dict = sort_rankdata(add_dict(userA_data, userB_data), "score", False)
    userAkey = userA["key"]
    userBkey = userB["key"]
    Arank = userA["username"] + "_rank"
    Brank = userB["username"] + "_rank"
    Ascore = userA["username"] + "_score"
    Bscore = userB["username"] + "_score"
    pd_aggregated = pd.DataFrame(columns = ["song", "mode", "level", userAkey, Arank, Ascore, userBkey, Brank, Bscore])
    
    pd_list = []
    pd_keys = []
    for item in aggregated_dict:
        #print(item)
        mode = "D"
        if item["mode"] == "Single": mode = "S"
        key = item["song"] + " " + mode + str(item["level"])
        if key not in pd_keys:
            pd_list.append(create_dict(item, userA, userB))
            pd_keys.append(key)
        else:
            for occupied in pd_list:
                #print(item)
                if occupied["key"] == key:
                    if occupied[Ascore] == 0:
                        occupied[Arank] = item["rank"]
                        occupied[Ascore] = item["score"]
                    else:
                        occupied[Brank] = item["rank"]
                        occupied[Bscore] = item["score"]
    pd_aggregated = pd.DataFrame(pd_list)


    winner = []
    for item in pd_list:
        if item[Ascore] > item[Bscore]:
            winner.append(userAkey)
        elif item[Ascore] < item[Bscore]:
            winner.append(userBkey)
        else:
            winner.append("DRAW")
    pd_aggregated["Winner"] = winner
    pd_aggregated = pd_aggregated.drop("key", axis = 1)
    
    pd_aggregated = pd_aggregated.style.apply(color_rows, userAkey = userAkey, axis = 1)
    
    return pd_aggregated



def create_dict(item, userA, userB):
    userAkey = userA["key"]
    userBkey = userB["key"]
    Arank = userA["username"] + "_rank"
    Brank = userB["username"] + "_rank"
    Ascore = userA["username"] + "_score"
    Bscore = userB["username"] + "_score"
    mode = "D"
    if item["mode"] == "Single": mode = "S"
    level_short = mode + str(item["level"])
    key = item["song"] + " " + level_short
    if item["username"] == userAkey[:-6]:
        return {"key":key, "song": item["song"], "level": level_short, Arank: item["rank"], Ascore: item["score"], Brank: "-", Bscore: 0}
    else:
        return {"key":key, "song": item["song"], "level": level_short, Arank: "-", Ascore: 0, Brank: item["rank"], Bscore: item["score"]}


def refine_solo_dataframe(dataframe_raw):
    levelmode = []
    #print(dataframe_raw)
    for index, row in dataframe_raw.iterrows():
        mode_short = "D"
        if row["mode"] == "Single": mode_short = "S"
        levelmode.append(mode_short+str(row["level"]))


    dataframe_raw["mode/level"] = levelmode

    dataframe_new = dataframe_raw[["song", "mode/level", "rank", "score"]]
    return dataframe_new
def rankdata(username, mode = "Full", levels = [20,28], songtype = songtype_all, version = version_all, sortme = "score", sort_all = False):
    data_user,count_user,total_user = rankdata_raw(username, mode = "Full", levels = [20,28], songtype = songtype_all, version = version_all)
    data_sorted = sort_rankdata(data_user, sortme, sort_all)
    data_sorted_pandas_raw = pd.DataFrame(data_sorted)
    data_sorted_pandas = refine_solo_dataframe(data_sorted_pandas_raw)
    achievement_rate = {}
    for item in count_user:
        if total_user[item] == 0: 
            achievement_rate[item] = 0
        else:
            achievement_rate[item] = round(count_user[item]/total_user[item],3)
    ranks = []
    for item in data_sorted:
        ranks.append(item["rank"])
    ranks.sort()
    ranks_by_level = {}
    for item in data_sorted:
        if item["level"] not in ranks_by_level:
            ranks_by_level[item["level"]] = [item["rank"]]
        else:
            ranks_by_level[item["level"]].append(item["rank"])
    for item in ranks_by_level:
        ranks_by_level[item].sort()
    return data_sorted_pandas, achievement_rate, ranks, ranks_by_level

def rankdata_compare_total_counts(userAcount, userBcount, userAdata, userBdata):
    userA_total_count = total_count(userAcount)
    userB_total_count = total_count(userBcount)
    userA_total_count_weighted = total_count_weighted(userAcount)
    userB_total_count_weighted = total_count_weighted(userBcount)
    userA_total_count_with_rank = total_count_with_rank(userAdata)
    userB_total_count_with_rank = total_count_with_rank(userBdata)
    userA_total_count_with_rank_weighted = total_count_with_rank_weighted(userAdata)
    userB_total_count_with_rank_weighted = total_count_with_rank_weighted(userBdata)
    return [[userA_total_count, userB_total_count],
            [userA_total_count_weighted, userB_total_count_weighted], 
            [userA_total_count_with_rank, userB_total_count_with_rank], 
            [userA_total_count_with_rank_weighted, userB_total_count_with_rank_weighted]]




def rankdata_compare(userA, userB):
    userA_raw = return_user_with_name(userA)
    userB_raw = return_user_with_name(userB)
    userAdata, userBdata, userAcount, userBcount, total_filtered = rankdata_compare_raw(userA, userB)
    userA_data_sorted, userA_achievement_rate, userA_ranks, userA_ranks_by_level = rankdata(userA)
    userB_data_sorted, userB_achievement_rate, userB_ranks, userB_ranks_by_level = rankdata(userB)
    user_comparison = [[userA_data_sorted, userB_data_sorted], [userA_achievement_rate, userB_achievement_rate], [userA_ranks, userB_ranks], [userA_ranks_by_level, userB_ranks_by_level]]
    total_count_comparison = rankdata_compare_total_counts(userAcount, userBcount, userAdata, userBdata)
    aggregated_dataframe = aggregate_dataframe(userAdata, userBdata, userA_raw, userB_raw)
    return user_comparison, total_count_comparison, aggregated_dataframe
#rankdata("naturenim", mode = "Single", levels=[21,23])

In [53]:
rankdata("naturenim")[0]
#rankdata_compare("naturenim", "EXC #7437")[0]

,song,mode/level,rank,score
0,ERRORCODE: 0,D27,9,931664
1,1950,D27,7,919509
2,Shub Sothoth,D27,10,908168
3,iRELLiA,D26,5,973275
4,Dement ~After Legend~,D26,8,969871
...,...,...,...,...
534,GLORIA,S20,70,994633
535,JANUS,S20,95,994601
536,Destination,S20,73,991136
537,Allegro furioso,S20,75,989599


In [26]:
'''
import pandas as pd
import numpy as np
abel = return_user_with_name("fefemz")
abel
hairda = return_user_with_name("frankeza")
hairda

abel_rankdata = filter_rankdata(abel,mode = "Double", levels = [25,27])
hairda_rankdata = filter_rankdata(hairda, mode = "Double", levels = [25,27])

def create_dict(item, userA, userB):
    userAkey = userA["key"]
    userBkey = userB["key"]
    Arank = userA["username"] + "_rank"
    Brank = userB["username"] + "_rank"
    Ascore = userA["username"] + "_score"
    Bscore = userB["username"] + "_score"
    mode = "D"
    if item["mode"] == "Single": mode = "S"
    level_short = mode + str(item["level"])
    key = item["song"] + " " + level_short
    if item["username"] == userAkey[:-6]:
        return {"key":key, "song": item["song"], "level": level_short, Arank: item["rank"], Ascore: item["score"], Brank: "-", Bscore: 0}
    else:
        return {"key":key, "song": item["song"], "level": level_short, Arank: "-", Ascore: 0, Brank: item["rank"], Bscore: item["score"]}


def color_rows(row, userAkey):
    # Use RGBA for semi-transparent colors (the last number is the alpha value)
    # The alpha value can be adjusted between 0 (fully transparent) and 1 (fully opaque)
    # Here we set it to 0.5 to blend the color with the white background, making it lighter
    color = 'rgba(125, 50, 50, 0.5)' if row['Winner'] == userAkey else 'rgba(50,50,125, 0.5)'
    return ['background-color: ' + color] * len(row)

def aggregate_dataframe(userA_data, userB_data, userA, userB):
    aggregated_dict = sort_rankdata(add_dict(userA_data, userB_data), "score", False)
    userAkey = userA["key"]
    userBkey = userB["key"]
    Arank = userA["username"] + "_rank"
    Brank = userB["username"] + "_rank"
    Ascore = userA["username"] + "_score"
    Bscore = userB["username"] + "_score"
    pd_aggregated = pd.DataFrame(columns = ["song", "mode", "level", userAkey, Arank, Ascore, userBkey, Brank, Bscore])
    
    pd_list = []
    pd_keys = []
    for item in aggregated_dict:
        #print(item)
        mode = "D"
        if item["mode"] == "Single": mode = "S"
        key = item["song"] + " " + mode + str(item["level"])
        if key not in pd_keys:
            pd_list.append(create_dict(item, userA, userB))
            pd_keys.append(key)
        else:
            for occupied in pd_list:
                #print(item)
                if occupied["key"] == key:
                    if occupied[Ascore] == 0:
                        occupied[Arank] = item["rank"]
                        occupied[Ascore] = item["score"]
                    else:
                        occupied[Brank] = item["rank"]
                        occupied[Bscore] = item["score"]
    pd_aggregated = pd.DataFrame(pd_list)


    winner = []
    for item in pd_list:
        if item[Ascore] > item[Bscore]:
            winner.append(userAkey)
        elif item[Ascore] < item[Bscore]:
            winner.append(userBkey)
        else:
            winner.append("DRAW")
    pd_aggregated["Winner"] = winner
    pd_aggregated = pd_aggregated.drop("key", axis = 1)
    pd_aggregated = pd_aggregated.style.apply(color_rows, userAkey = userAkey, axis=1)

    return pd_aggregated

aggregate_dataframe(abel_rankdata, hairda_rankdata, abel, hairda)
'''

,song,level,FEFEMZ_rank,FEFEMZ_score,FRANKEZA_rank,FRANKEZA_score,Winner
0,ERRORCODE: 0,D27,1,981608,2,972864,FEFEMZ #1489
1,Shub Sothoth,D27,2,969275,1,976429,FRANKEZA #9606
2,1950,D27,1,966236,2,957965,FEFEMZ #1489
3,Neo Catharsis,D27,1,965411,-,0,FEFEMZ #1489
4,KUGUTSU,D27,1,965334,2,954437,FEFEMZ #1489
5,Ultimatum,D27,1,962065,-,0,FEFEMZ #1489
6,Aragami,D26,1,993299,2,983395,FEFEMZ #1489
7,Repentance,D26,1,992706,2,987620,FEFEMZ #1489
8,BEMERA,D26,1,992143,2,982123,FEFEMZ #1489
9,Prime Time,D26,1,991688,2,987121,FEFEMZ #1489


In [18]:
print(search_user("dulki"))

dulki = return_user_with_name("dulki")

print(dulki)

filter_rankdata(dulki, mode = "Full", levels = [20,26])

{'DULKI': [['#2827', {20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 2, 26: 0, 27: 0, 28: 0}]]}
{'key': 'DULKI #2827', 'username': 'DULKI', 'userID': '#2827', 'single_data': {'20': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]], '21': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]], '22': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]], '23': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]], '24': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]], '25': [[[{'song': 'KUGUTSU', 'mode': 'Single', 'level': 25, 'username': 'DULKI', 'userid': '#2827', 'score': 859932, 'rank': 54, 'type': 'Arcade', 'version': 'PHOENIX'}, {'song': 'Neo Catharsis', 'mode': 'Single', 'level': 25, 'username': 'DULKI', 'userid': '#2827', 'score': 894512, 'rank': 87, 'type': 'Arcade', 'version': 'PHOENIX'}], [], []], [[], [], []], [[], [], []], [[], [], []]], '26': [[[], [], []], [[], [], []], [[], [], []], [[], [], []]]}, 'single_counts': {'20': [[0, 0, 0], [0, 0, 0], [0, 0, 0], 

{20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [{'song': 'KUGUTSU',
   'mode': 'Single',
   'level': 25,
   'username': 'DULKI',
   'userid': '#2827',
   'score': 859932,
   'rank': 54,
   'type': 'Arcade',
   'version': 'PHOENIX'},
  {'song': 'Neo Catharsis',
   'mode': 'Single',
   'level': 25,
   'username': 'DULKI',
   'userid': '#2827',
   'score': 894512,
   'rank': 87,
   'type': 'Arcade',
   'version': 'PHOENIX'}],
 26: []}

In [20]:
import pandas as pd

data_sample = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', "Kendrik"],
    'Score': [82, 91, 78, 85, 72]
}
df = pd.DataFrame(data_sample)

# Function to apply color based on condition
def color_rows(row):
    # Use RGBA for semi-transparent colors (the last number is the alpha value)
    # The alpha value can be adjusted between 0 (fully transparent) and 1 (fully opaque)
    # Here we set it to 0.5 to blend the color with the white background, making it lighter
    color = 'rgba(255, 125, 125, 0.9)' if row['Score'] < 80 else 'rgba(125,125,255, 0.9)'
    return ['background-color: ' + color] * len(row)
styled_df = df.style.apply(color_rows, axis=1)

# To visualize the DataFrame in a Python script:
#print(styled_df.render())  # This will print the HTML of the styled DataFrame
styled_df

,Name,Score
0,Alice,82
1,Bob,91
2,Charlie,78
3,David,85
4,Kendrik,72


In [158]:
# Initialize an empty list to collect dictionaries
data_list = []

# Simulate adding dictionaries line by line
data_list.append({'song': 'Jupin', 'artist': 'PABEL', 'score': 977285})
data_list.append({'song': 'Gargoyle - FULL SONG -', 'artist': 'PABEL', 'score': 974987})
data_list.append({'song': 'Solfeggietto', 'artist': 'PABEL', 'score': 969867})

# Once all data is collected, convert the list to a DataFrame
df = pd.DataFrame(data_list)

print(df)

                     song artist   score
0                   Jupin  PABEL  977285
1  Gargoyle - FULL SONG -  PABEL  974987
2            Solfeggietto  PABEL  969867
